In [1]:
import pandas as pd
import requests
import json
import sys
import datetime

In [2]:
api_key = 'YOUR OWN API KEY HERE'
api_key

'47949907-3ad8-4da3-b79a-73ccf147d793'

In [3]:
def fetch_entry_from_id(manager, sample_id):
    session = requests.Session()
    url = "https://api.scispot.io/tryingtofixcors/labsheets/find-row-by-id"
    payload = {
        "apiKey": api_key,
        "labsheet": manager,
        "id": sample_id
    }
    ret = session.post(url, json=payload)
    return json.loads(ret.text)

def json_to_pd_df(json_in):
    df_out = pd.DataFrame(data=json_in['rows'])#.T
    df_out.columns = list(json_in['headers'])
    return df_out

In [36]:
ag_test_samp = fetch_entry_from_id("Sample Manager", "ag_test_sample")
#print(ag_test_samp)
json_to_pd_df(ag_test_samp)

,UUID,Sample ID,Working compound,Working cell line,Pert Dose,Pert Time,Pert Protocol,Tissue ID,Tissue Passage,Type,Parent,Children,Location,Storage Form,Storage Amount,Pass,Quantity Number of Cells (x1000),Created By
0,8ee0b10d-db00-46ac-a6d4-a52388f881ef,ag_test_sample,Biotin,THP-1,,,,,,,daa8f782-9339-47e8-82af-ef1238822c13,,,,,,3000,


In [5]:
#fetch required rows for protein manager new entry 
def fetch_random_entry():
    session = requests.Session()
    url = "https://api.scispot.io/tryingtofixcors/labsheets/list-rows"
    payload = {
        "apiKey": api_key,
        "manager": "Peptide Manager",
        "pageSize": "1",
        "page": "1"
    }
    ret = session.post(url, json=payload)
    return json.loads(ret.text)

In [6]:
pro_manager_rows_required = pd.DataFrame(data = fetch_random_entry()['rows'], 
                                         columns = fetch_random_entry()['headers'])
#print(pro_manager_rows_required.columns)
pro_manager = pro_manager_rows_required.columns
print(pro_manager)

Index(['UUID', 'Digestion ID', 'Protein ID', 'Digestion Date', 'Protocol ID',
       'Digestion Scientist', 'Protein Input', 'Reduction Chemical',
       'Reduction Dose', 'Alkylation Chemical', 'Alkylation Dose',
       'Protease Chemical', 'Protease Dose', 'Quench Chemical', 'Quench Dose',
       'Storage Location', 'Storage Form', 'Storage Concentration', 'Pass',
       'Parent', 'Children', 'Location', 'Created By'],
      dtype='object')


In [7]:
def get_table(table_name):
    url = "https://api.scispot.io/tryingtofixcors/labsheets/list-rows"
    request_payload = {
        "apiKey": api_key,
        "manager": table_name,
        "page": "1",
        "pageSize": "50"
    }
    x = requests.post(url, json = request_payload)
    ret = json.loads(x.text)
    return ret 

In [37]:
protein_manager = get_table("Protein Manager")
json_to_pd_df(protein_manager).head(10)

,UUID,Protein ID,Sample ID,Protocol ID,Extraction Date,Extraction Scientist,Extraction Fraction,Extraction Notes,Protein Concentration (ug/mL),BCA Results,Location,Storage Form,Storage Amount (uL),Pass,Parent,Children,Created By,Creation Date
0,c6d7bd0f-2ad9-443c-ae7d-386b2f1c8d09,ag_test_sample_chromatin,ag_test_sample,ag_test_sample,07/05/2023,Andrea,chromatin,sample notes go here,0,bca results here,,storage form,storage amount uL,True,,,,
1,8da8f096-d7be-48f7-9928-724cc4258dd4,230516_HepG2_IL6_Jakafi_01_chrom,,,05/16/2023,Andrea,Chromatin,,,,,,750,True,54365b2b-b39a-4b42-9c48-76f903ec24b6,"e64d8da3-11f6-4555-a10c-e26875c1109f, 37f64d0c...",,
2,c4b847f2-61b3-4678-bc78-37282aceddd2,230516_HepG2_IL6_Jakafi_02_insol,,,05/16/2023,Andrea,Insoluble,,,,,,750,True,1bda0537-7c37-4f26-8795-b4eb1f7669f6,"18813881-1c66-4ec7-9874-f4a584f0e04f, 1b0ac77d...",,
3,997885d5-7ac0-4c6c-b9bf-e3a2ba7d9ae1,230516_HepG2_IL6_Jakafi_02_nuc,,,05/16/2023,Andrea,Nuclear,,,,,,750,True,1bda0537-7c37-4f26-8795-b4eb1f7669f6,"6ab80672-0c74-4f3b-bb2f-8b6759a9187e, ed40a2cd...",,
4,1b55b4f0-cda3-4d2a-b299-ee0a3d106e63,230516_HepG2_IL6_Jakafi_03_insol,,,05/16/2023,Andrea,Insoluble,,,,,,750,True,c35bb3b5-7a2e-4b43-b175-3bdd69bbed34,"62995e9f-a683-4654-8ddf-703bd49097af, 448a40b4...",,
5,303b2def-0b4c-4206-8edd-3f77761ae28b,230516_HepG2_IL6only_03_nuc,,,05/16/2023,Andrea,Nuclear,,,,,,750,True,edfa0fdd-01a1-4486-badf-14a5eb083638,"0ff0e92d-445b-4704-95a5-e17ea70cd364, 6fe8fff7...",,
6,d00dee48-5d3e-4795-afc6-8e19970b9331,230516_HepG2_DMSO_01_insol,,,05/16/2023,Andrea,Insoluble,,,,,,750,True,19225f0e-73e6-4028-bfe7-ebba58f99855,"40ccfc05-c18d-4201-b293-681fa2c70e37, 645794c6...",,
7,c12ba3f5-be15-4115-b74d-4f92d391aded,230516_HepG2_DMSO_01_nuc,,,05/16/2023,Andrea,Nuclear,,,,,,750,True,19225f0e-73e6-4028-bfe7-ebba58f99855,"aa13be06-ba59-4219-9fcb-a561b05f6f49, a5abf137...",,
8,6b68dc8c-49d8-46d2-9909-9233fa99cd88,230516_HepG2_DMSO_02_nuc,,,05/16/2023,Andrea,Nuclear,,,,,,750,True,0cb1a11c-4a66-4990-b903-82f4e84e543d,"e8e70afd-02ac-4edb-8ec4-8f70e1cc1c48, 369e1521...",,
9,82250d7e-14f7-4938-8149-84fb1dfd1d29,230516_HepG2_IL6_Jakafi_03_chrom,,,05/16/2023,Andrea,Chromatin,,,,,,750,True,c35bb3b5-7a2e-4b43-b175-3bdd69bbed34,"7ed87ccc-fcda-4866-aaa6-a530f3245cde, bed36dc8...",,


In [9]:
def find_entry_from_uuid(manager, uuid_list):
    session = requests.Session()
    url = "https://api.scispot.io/tryingtofixcors/labsheets/find-row"
    payload = {
        "apiKey": api_key,
        "manager": manager,
        "uuid": uuid_list
    }
    ret = session.post(url, json=payload)
    return json.loads(ret.text)

In [10]:
#entry = find_entry_from_uuid("Sample Manager", "8ee0b10d-db00-46ac-a6d4-a52388f881ef")
test_sample = json_to_pd_df(ag_test_samp)
test_sample
#entry

,UUID,Sample ID,Working compound,Working cell line,Pert Dose,Pert Time,Pert Protocol,Tissue ID,Tissue Passage,Type,Parent,Children,Location,Storage Form,Storage Amount,Pass,Quantity Number of Cells (x1000),Created By
0,8ee0b10d-db00-46ac-a6d4-a52388f881ef,ag_test_sample,Biotin,THP-1,,,,,,,daa8f782-9339-47e8-82af-ef1238822c13,,,,,,3000,


In [11]:
sample_id = test_sample['Sample ID'][0]
sample_id

'ag_test_sample'

In [12]:
#UUID	Protein ID	Sample ID	Protocol ID	Extraction Date	Extraction Scientist	
#Extraction Fraction	Extraction Notes	Protein Concentration (ug/mL)	BCA Results	Location	
#Storage Form	Storage Amount (uL)	Pass	Parent	Children	Created By	Creation Date
def add_protein_manager_entry_row(new_row):
    url = "https://api.scispot.io/v2/labsheets/add-rows" #https://api.scispot.io/tryingtofixcors/labsheets/add-rows
    payload = {
        "apiKey": api_key,
        "manager": "Protein Manager",
        "rows": [new_row]
    }
    headers = {"Content-Type": "application/json"}
    #print(new_row)
    
    response = requests.post(url, json=payload, headers=headers)
    print(response.text)

def new_protein_row(sample_uuid, fraction):
    protein_id = sample_id + "_" + fraction
    
    new_protein_row = [
        protein_id,
        sample_uuid,
        "8ee0b10d-db00-46ac-a6d4-a52388f881ef", #uuid
        "07/05/2023",
        "Andrea",
        fraction,
        "sample notes go here",
        "0", #ug/mL protein concentration
        "bca results here", #bca results
        "", #Location
        "storage form", #storage form
        "storage amount uL", #storage amt 
        "True", #Pass?
        sample_uuid, #parent uuid
        "x", #children uuid
        "y", #created by
        "" #creation date
    ]
    return(new_protein_row)

In [13]:
new_row = new_protein_row("8ee0b10d-db00-46ac-a6d4-a52388f881ef", "chromatin")
new_row

['ag_test_sample_chromatin',
 '8ee0b10d-db00-46ac-a6d4-a52388f881ef',
 '8ee0b10d-db00-46ac-a6d4-a52388f881ef',
 '07/05/2023',
 'Andrea',
 'chromatin',
 'sample notes go here',
 '0',
 'bca results here',
 '',
 'storage form',
 'storage amount uL',
 'True',
 '8ee0b10d-db00-46ac-a6d4-a52388f881ef',
 'x',
 'y',
 '']

In [14]:
add_protein_manager_entry_row(new_row)

[{"row":"1","success":"true","uuid":"c6d7bd0f-2ad9-443c-ae7d-386b2f1c8d09"}]


In [15]:
test_pro = fetch_entry_from_id("Protein Manager", "ag_test_sample_chromatin")
test_protein = json_to_pd_df(test_pro)
test_protein

,UUID,Protein ID,Sample ID,Protocol ID,Extraction Date,Extraction Scientist,Extraction Fraction,Extraction Notes,Protein Concentration (ug/mL),BCA Results,Location,Storage Form,Storage Amount (uL),Pass,Parent,Children,Created By,Creation Date
0,c6d7bd0f-2ad9-443c-ae7d-386b2f1c8d09,ag_test_sample_chromatin,ag_test_sample,ag_test_sample,07/05/2023,Andrea,chromatin,sample notes go here,0,bca results here,,storage form,storage amount uL,True,,,,


In [16]:
pep_manager_rows_required = pd.DataFrame(data = fetch_random_entry()['rows'], 
                                         columns = fetch_random_entry()['headers'])
#print(pro_manager_rows_required.columns)
pep_manager = pep_manager_rows_required.columns
print(pep_manager)

Index(['UUID', 'Digestion ID', 'Protein ID', 'Digestion Date', 'Protocol ID',
       'Digestion Scientist', 'Protein Input', 'Reduction Chemical',
       'Reduction Dose', 'Alkylation Chemical', 'Alkylation Dose',
       'Protease Chemical', 'Protease Dose', 'Quench Chemical', 'Quench Dose',
       'Storage Location', 'Storage Form', 'Storage Concentration', 'Pass',
       'Parent', 'Children', 'Location', 'Created By'],
      dtype='object')


In [42]:
protein_id = test_protein['Protein ID'][0]
print(protein_id)
protein_uuid_ag = test_protein['UUID'][0]
protein_uuid_ag

ag_test_sample_chromatin


'c6d7bd0f-2ad9-443c-ae7d-386b2f1c8d09'

In [69]:
def add_peptide_manager_entry(new_pep_row):
    url = "https://api.scispot.io/tryingtofixcors/labsheets/add-rows"
    #url = "https://api.scispot.io/v2/labsheets/add-rows"
    payload = {
        "apiKey": api_key,
        "manager": "Peptide Manager",
        "rows": [new_pep_row]
    }
    headers = {"Content-Type": "application/json"}
        
    response = requests.request("POST", url, json=payload, headers=headers)
    print(response.text)

#required order/required peptide manager columns:     
#Digestion ID,Protein ID,Digestion Date,Protocol ID,Digestion Scientist,Protein Input,
#Reduction Chemical,Reduction Dose,Alkylation Chemical,Alkylation Dose,Protease Chemical,
#Protease Dose,Quench Chemical,Quench Dose,Storage Location,Storage Form,Storage Concentration,
#Pass,Parent,Children,Location,Created By

def new_peptide_row(protein_uuid):
    peptide_id = protein_id + "_peptide" 
    
    new_peptide_row = [{
        "UUID" : protein_uuid_ag,
        "Digestion ID" : peptide_id, 
        "Protein ID" : protein_uuid_ag,
        "Digestion Date" : "07/06/2023",
        "Protocol ID" : "8ee0b10d-db00-46ac-a6d4-a52388f881ef",
        "Digestion Scientist" : "Andrea",
        "Protein Input" : "protein input",
        "Reduction Chemical" : "DTT",
        "Reduction Dose" : "4 mM",
        "Alkylation Chemical" : "IAM",
        "Alkylation Dose" : "20 mM",
        "Protease Chemical" : "Trypsin",
        "Protease Dose" : "1-10",
        "Quench Chemical" : "Formic Acid",
        "Quench Dose" : "20 uL",
        "Storage Location" : "-80 #1",
        "Storage Form" : "Storage Form",
        "Storage Concentration" : "storage conc",
        "Pass" : "True",
        "Parent" : protein_uuid_ag,
        "Children" : "",
        "Location" : "",
        "Created By" : "Andrea"
    }]
    
    return(new_peptide_row)
sys.stdout.write("Imported functions for manipulating Peptide Manager")

Imported functions for manipulating Peptide Manager

51

In [49]:
print(protein_uuid_ag)

c6d7bd0f-2ad9-443c-ae7d-386b2f1c8d09


In [70]:
new_peptide = new_peptide_row(protein_uuid_ag)
new_peptide

[{'UUID': 'c6d7bd0f-2ad9-443c-ae7d-386b2f1c8d09',
  'Digestion ID': 'ag_test_sample_chromatin_peptide',
  'Protein ID': 'c6d7bd0f-2ad9-443c-ae7d-386b2f1c8d09',
  'Digestion Date': '07/06/2023',
  'Protocol ID': '8ee0b10d-db00-46ac-a6d4-a52388f881ef',
  'Digestion Scientist': 'Andrea',
  'Protein Input': 'protein input',
  'Reduction Chemical': 'DTT',
  'Reduction Dose': '4 mM',
  'Alkylation Chemical': 'IAM',
  'Alkylation Dose': '20 mM',
  'Protease Chemical': 'Trypsin',
  'Protease Dose': '1-10',
  'Quench Chemical': 'Formic Acid',
  'Quench Dose': '20 uL',
  'Storage Location': '-80 #1',
  'Storage Form': 'Storage Form',
  'Storage Concentration': 'storage conc',
  'Pass': 'True',
  'Parent': 'c6d7bd0f-2ad9-443c-ae7d-386b2f1c8d09',
  'Children': '',
  'Location': '',
  'Created By': 'Andrea'}]

In [71]:
add_peptide_manager_entry(new_peptide)

{"timestamp":"2023-07-10T23:16:25.408+0000","status":400,"error":"Bad Request","message":"","path":"/labsheets/add-rows"}


In [59]:
new_row_pep = new_peptide_row(protein_uuid_ag)
new_row_pep
add_peptide_manager_entry(new_row_pep)
print(new_row_pep)

{"timestamp":"2023-07-10T22:53:27.102+0000","status":400,"error":"Bad Request","message":"","path":"/labsheets/add-rows"}
[{'UUID': 'c6d7bd0f-2ad9-443c-ae7d-386b2f1c8d09', 'Digestion ID': 'ag_test_sample_chromatin_peptide', 'Protein ID': 'c6d7bd0f-2ad9-443c-ae7d-386b2f1c8d09', 'Digestion Date': '07/06/2023', 'Protocol ID': '8ee0b10d-db00-46ac-a6d4-a52388f881ef', 'Digestion Scientist': 'Andrea', 'Protein Input': 'protein input', 'Reduction Chemical': 'DTT', 'Reduction Dose': '4 mM', 'Alkylation Chemical': 'IAM', 'Alkylation Dose': '20 mM', 'Protease Chemical': 'Trypsin', 'Protease Dose': '1-10', 'Quench Chemical': 'Formic Acid', 'Quench Dose': '20 uL', 'Storage Location': '-80 #1', 'Storage Form': 'Storage Form', 'Storage Concentration': 'storage conc', 'Pass': 'True', 'Parent': 'c6d7bd0f-2ad9-443c-ae7d-386b2f1c8d09', 'Children': '', 'Location': '', 'Created By': 'Andrea'}]


In [ ]:
Index(['UUID', 'Digestion ID', 'Protein ID', 'Digestion Date', 'Protocol ID',
       'Digestion Scientist', 'Protein Input', 'Reduction Chemical',
       'Reduction Dose', 'Alkylation Chemical', 'Alkylation Dose',
       'Protease Chemical', 'Protease Dose', 'Quench Chemical', 'Quench Dose',
       'Storage Location', 'Storage Form', 'Storage Concentration', 'Pass',
       'Parent', 'Children', 'Location', 'Created By'],
      dtype='object')